In [ ]:
# Imports
import os
import numpy as np
import time 
import h5py
import cyborg_vector_search_py as cvs

In [ ]:
# Parameters
dataset = "sift-128-euclidean"
dimension = 128
index_type = "IVFFlat"
n_lists = 4096
metric = "euclidean"

# Training parameters
batch_size = 2048
max_iters = 100
max_dataset_size = 1000000
training_ratio = 0.1 # Number of vectors to use for training (rest will be indexed post-training)
top_k = 100

# Location configuration
redis_connection_string = "host:127.0.0.1,port:6379,db:0" # Set this to your Redis connection string
index_location = cvs.LocationConfig(cvs.Location.REDIS, "index_table", redis_connection_string)
config_location = cvs.LocationConfig(cvs.Location.REDIS, "config_table", redis_connection_string)

# Index configuration
index_config = cvs.IndexIVFFlat(dimension, n_lists, metric, False)

In [ ]:
# Load the dataset
with h5py.File(f'data/{dataset}.hdf5', 'r') as file:
    train = np.array(file['train'], dtype=np.float32)
    test = np.array(file['test'], dtype=np.float32)
    neighbors = np.array(file['neighbors'], dtype=np.int32)

# Setup the training data
vectors = train[:max_dataset_size]
ids = np.arange(max_dataset_size)
training_size = int(max_dataset_size * training_ratio)
training_vectors = vectors[:training_size]
training_ids = ids[:training_size]
remaining_vectors = vectors[training_size:]
remaining_ids = ids[training_size:]

In [ ]:
# Setup the CVS client
client = cvs.CyborgVectorSearch(
    index_location=index_location,
    config_location=config_location,
    cpu_threads = 16,
    gpu_accelerate = False
)

# Dummy index name and key
index_name = "redis_example_index"
index_key = bytes([0] * 32)

# Delete example index if it exists already
try:
    client.load_index(index_name, index_key)
    client.delete_index()
except:
    pass # Index does not exist, no need to delete

# Create the index
client.create_index(index_name, index_key, index_config)

In [ ]:
# Upsert untrained

start = time.time()
client.upsert(training_vectors, training_ids)
print(f"Upserted {training_size} vectors in {time.time() - start:.2f} seconds")

In [ ]:
# Untrained query

n_probes = 10
num_queries = 1000
initial_queries = test[:num_queries]

start = time.time()
results = client.query(initial_queries, top_k, n_probes)
end = time.time()

# Compute the neighbors for the training_size vectors
dists = np.sum(training_vectors**2, axis=1) - 2 * np.dot(initial_queries, training_vectors.T) + np.sum(initial_queries**2, axis=1)[:, np.newaxis]
initial_neighbors = np.argpartition(dists, top_k, axis=1)[:, :top_k]
initial_neighbors = np.take_along_axis(initial_neighbors, np.argsort(dists[np.arange(dists.shape[0])[:, None], initial_neighbors], axis=1), axis=1)

# Compute the recall using the neighbors
recall = np.zeros(initial_queries.shape[0])
for i in range(initial_queries.shape[0]):
    recall[i] = len(np.intersect1d(initial_neighbors[i], results[i])) / len(initial_neighbors[i])

print(f"Queried {initial_queries.shape[0]} vectors in {end - start:.2f} seconds")
print(f"QPS: {num_queries / (end - start):.2f}")
print(f"Mean recall: {recall.mean() * 100:.2f}%")

In [ ]:
# Train index for faster queries

start = time.time()
client.train_index(batch_size, max_iters)
print(f"Trained index with {training_size} vectors in {time.time() - start:.2f} seconds")

In [ ]:
# Trained query

n_probes = 32

# Query the test set
start = time.time()
results = client.query(initial_queries, top_k, n_probes)
end = time.time()

# Compute the recall using the neighbors
results = np.array(results)
recall = np.zeros(initial_queries.shape[0])
for i in range(initial_queries.shape[0]):
    recall[i] = len(np.intersect1d(initial_neighbors[i], results[i])) / len(initial_neighbors[i])

print(f"Queried {initial_queries.shape[0]} vectors in {end - start:.2f} seconds")
print(f"QPS: {num_queries / (end - start):.2f}")
print(f"Mean recall: {recall.mean() * 100:.2f}%")

In [ ]:
# Upsert remaining (skip if training ratio is 1)
if training_ratio < 1:

    start = time.time()
    client.upsert(remaining_vectors, remaining_ids)
    print(f"Upserted {remaining_vectors.shape[0]} vectors in {time.time() - start:.2f} seconds")
    print(f"VPS: {remaining_vectors.shape[0] / (time.time() - start):.2f}")

In [ ]:
# Trained query

n_probes = 32

# Query the test set
start = time.time()
results = client.query(test, top_k, n_probes)
end = time.time()

# Convert the results to numpy array
results = np.array(results)
if neighbors.shape != results.shape:
    raise ValueError("The shapes of the neighbors and results do not match")

# Compute the recall using the neighbors
recall = np.zeros(test.shape[0])
for i in range(test.shape[0]):
    recall[i] = len(np.intersect1d(neighbors[i], results[i])) / len(neighbors[i])

print(f"Queried {test.shape[0]} vectors in {end - start:.2f} seconds")
print(f"QPS: {test.shape[0] / (end - start):.2f}")
print(f"Mean recall: {recall.mean() * 100:.2f}%")

In [ ]:
# Cleanup

client.delete_index()